In [ ]:
# Explore the Event Rates in SBND

In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#MC CV Sample --> High Stats
#infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats.root"
infile = "../NTuples/mc_production_true_particles_ntuple_v2.root"

In [ ]:
inFile = uproot.open(infile)

inFileROOT = ROOT.TFile.Open(infile, "READ")
#h_tot_pot = inFileROOT.Get("TotalPOT")
h_tot_pot = inFileROOT.Get("TOTPOT_Clone")
TOT_POT = h_tot_pot.GetBinContent(1)
inFileROOT.Close()
TOT_POT = f"{TOT_POT:.2e}"
print("Total POT", TOT_POT)

#slc_tree = inFile["slc_truth_tree"]
part_tree1 = inFile["part_tree1"]
#part_tree2 = inFile["part_tree2"]
part_tree3 = inFile["part_tree3"]


part_df1 = part_tree1.arrays(part_tree1.keys(), library="pd")
#part_df2 = part_tree2.arrays(part_tree2.keys(), library="pd")
part_df3 = part_tree3.arrays(part_tree3.keys(), library="pd")


part_df1[:2]

In [ ]:
#part_df2[:2]

In [ ]:
part_df3[:2]

# Get the Neutrinos in Truth

In [ ]:
#part_df2["pdg"] = part_df1["pdg"]
part_df3["pdg"] = part_df1["pdg"]
part_df3["interaction_id"] = part_df1["interaction_id"]

nu_df1 = part_df1.query("(pdg == 12 or pdg == -12 or pdg == 14 or pdg == -14) and interaction_id != -1")
#nu_df2 = part_df2.query("pdg == 12 or pdg == -12 or pdg == 14 or pdg == -14")
nu_df3 = part_df3.query("(pdg == 12 or pdg == -12 or pdg == 14 or pdg == -14) and interaction_id != -1")

nu_df3[:2]

# All Neutrino Interaction IDs 

In [ ]:
int_ids = list(set(list(nu_df3["interaction_id"].values)))
print(int_ids)

In [ ]:
def is_AV(row):
    xs,ys,zs = row["start_x"], row["start_y"], row["start_z"]
    xe,ye,ze = row["end_x"], row["end_y"], row["end_z"]
    v = -9998
    if (xs > v and ys > v and zs > v and xe > v and ye > v and ze > v):
        return 1
    else:
        return 0

nu_df3["AV"] = nu_df3.apply(is_AV, axis=1)
nu_df3[:4]

# AV and Dirt Interaction IDs

In [ ]:
av_int_ids = list(set(list(nu_df3.query("AV == 1")["interaction_id"].values)))
dirt_int_ids = list(set(list(nu_df3.query("AV == 0")["interaction_id"].values)))
print(av_int_ids)
print(dirt_int_ids)

In [ ]:
plt.hist(nu_df3.query("AV == 1 and 0 <= startT <= 10")["startT"].values, bins=50, histtype="step", label="AV startT")
plt.hist(nu_df3.query("AV == 1 and 0 <= startT <= 10")["genT"].values, bins=50, histtype="step", label="AV genT")
plt.hist(nu_df3.query("AV == 0 and 0 <= genT <= 10")["genT"].values, bins=50, histtype="step", label="Dirt genT")
plt.xlabel("True Time Relative to Beam Spill [us]", fontsize=14)
plt.xlim([0, 10])
#plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
B = np.arange(0, 10, 0.1)
for num in av_int_ids:
    t = nu_df3.query("AV == 1 and 0 <= startT <= 10 and interaction_id == "+str(num))["startT"].values
    plt.hist(t, bins=B, histtype="step", label=str(num))

plt.xlabel("True Start Time Relative to Beam Spill [us]", fontsize=14)
plt.legend()
plt.yscale("log")
plt.title("AV Neutrino Interaction IDs", fontsize=14)
plt.show()

In [ ]:
#plt.hist(nu_df3.query("AV == 1 and 0 <= startT <= 10")["startT"].values, bins=50, histtype="step", label="AV startT")
plt.hist(nu_df3.query("AV == 1")["genT"].values, bins=50, histtype="step", label="AV genT")
plt.hist(nu_df3.query("AV == 1")["startT"].values, bins=50, histtype="step", label="AV startT")
plt.hist(nu_df3.query("AV == 0")["genT"].values, bins=50, histtype="step", label="Dirt")
#plt.xlim([0, 10])
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
plt.hist(nu_df3.query("AV == 1")["endT"].values, bins=50, histtype="step", label="AV endT")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
plt.hist(nu_df3.query("AV == 1 and 0<= endT <= 10")["endT"].values, bins=B, histtype="step", label="AV endT")
plt.plot([1.6, 1.6], [0.00001, 10**4], c="r", label="1.6 us")
plt.plot([2, 2], [0.00001, 10**4], c="g", label="2 us")
#plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
x = nu_df3.query("AV == 1 and 0<= endT <= 10")["end_x"].values
y = nu_df3.query("AV == 1 and 0<= endT <= 10")["end_y"].values

#plt.hist2d(x, y, bins=100, cmap)

plt.title("AV Neutrinos (0 <= endT <= 10 us)", fontsize=20)
cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
cmap.set_under(color='white')  
hist = plt.hist2d(x, y, bins=50, cmap=cmap)
cbar = plt.colorbar(hist[3], label='Counts')
plt.xlabel("End X [cm]", fontsize=14)
plt.ylabel("End Y [cm]", fontsize=14)
#plt.xlim([min(x), max(x)])
#plt.ylim([min(x), max(x)])
#plt.savefig(fig_dir+"heatmap_north_after.png", format='png')
plt.show()

print("Max X", max(x), "Min X", min(x))
print("Max Y", max(y), "Min Y", min(y))

In [ ]:
z = nu_df3.query("AV == 1 and 0<= endT <= 10")["end_z"].values
y = nu_df3.query("AV == 1 and 0<= endT <= 10")["end_y"].values

#plt.hist2d(x, y, bins=100, cmap)

plt.title("AV Neutrinos (0 <= endT <= 10 us)", fontsize=20)
cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
cmap.set_under(color='white')  
hist = plt.hist2d(z, y, bins=50, cmap=cmap)
cbar = plt.colorbar(hist[3], label='Counts')
plt.xlabel("End Z [cm]", fontsize=14)
plt.ylabel("End Y [cm]", fontsize=14)
#plt.xlim([min(x), max(x)])
#plt.ylim([min(x), max(x)])
#plt.savefig(fig_dir+"heatmap_north_after.png", format='png')
plt.show()

print("Max Z", max(z), "Min Z", min(z))

In [ ]:
def is_TPC(row):
    xe,ye,ze = row["end_x"], row["end_y"], row["end_z"]
    xe, ye = abs(xe), abs(ye)
    
    if (xe <= 200 and ye <= 200 and 0 <= ze <= 500):
        return 1
    else:
        return 0

nu_df3["isTPC"] = nu_df3.apply(is_TPC, axis=1)
nu_df3[:3]

In [ ]:
x = nu_df3.query("isTPC == 1 and 0<= endT <= 10")["end_x"].values
y = nu_df3.query("isTPC == 1 and 0<= endT <= 10")["end_y"].values

#plt.hist2d(x, y, bins=100, cmap)

plt.title("TPC Neutrinos (0 <= endT <= 10 us)", fontsize=20)
cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
cmap.set_under(color='white')  
hist = plt.hist2d(x, y, bins=50, cmap=cmap)
cbar = plt.colorbar(hist[3], label='Counts')
plt.xlabel("End X [cm]", fontsize=14)
plt.ylabel("End Y [cm]", fontsize=14)
#plt.xlim([min(x), max(x)])
#plt.ylim([min(x), max(x)])
#plt.savefig(fig_dir+"heatmap_north_after.png", format='png')
plt.show()

In [ ]:
z = nu_df3.query("isTPC == 1 and 0<= endT <= 10")["end_z"].values
y = nu_df3.query("isTPC == 1 and 0<= endT <= 10")["end_y"].values

#plt.hist2d(x, y, bins=100, cmap)

plt.title("TPC Neutrinos (0 <= endT <= 10 us)", fontsize=20)
cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
cmap.set_under(color='white')  
hist = plt.hist2d(z, y, bins=50, cmap=cmap)
cbar = plt.colorbar(hist[3], label='Counts')
plt.xlabel("End Z [cm]", fontsize=14)
plt.ylabel("End Y [cm]", fontsize=14)
#plt.xlim([min(x), max(x)])
#plt.ylim([min(x), max(x)])
#plt.savefig(fig_dir+"heatmap_north_after.png", format='png')
plt.show()

In [ ]:
plt.hist(nu_df3.query("isTPC == 1 and 0 <= endT <= 10")["endT"].values, bins=B, histtype="step", label="TPC")
plt.hist(nu_df3.query("isTPC == 0 and 0 <= genT <= 10")["genT"].values, bins=B, histtype="step", label="Not TPC")
plt.hist(nu_df3.query("isTPC == 0 and AV == 1 and (0 <= endT <= 10)")["endT"].values, 
         bins=B, histtype="step", label="Not TPC and AV")
plt.hist(nu_df3.query("AV == 0 and isTPC == 0 and 0 <= genT <= 10")["genT"].values, 
         bins=B, histtype="step", label="Not AV and Not TPC")
#plt.plot([1.6, 1.6], [0.00001, 10**4], c="r", label="1.6 us")
#plt.plot([2, 2], [0.00001, 10**4], c="g", label="2 us")
plt.xlabel("True Time Since the Beam Spill [us]", fontsize=14)
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
B = np.arange(0, 10, 0.1)
for num in dirt_int_ids:
    t = nu_df3.query("isTPC == 0 and 0 <= genT <= 10 and interaction_id == "+str(num))["genT"].values
    plt.hist(t, bins=B, histtype="step", label=str(num))

plt.xlabel("True Gen Time Relative to Beam Spill [us]", fontsize=14)
plt.legend()
plt.yscale("log")
plt.title("Dirt Neutrino Interaction IDs", fontsize=14)
plt.show()

In [ ]:
nu_df1[:5]

In [ ]:
nu_df1["isTPC"] = nu_df3["isTPC"]
nu_df1[:2]

In [ ]:

def num_d(row):

    N_inter = 0
    #for num in range(nu_df1.query("isTPC == 1").shape[0]):

    #nu_ex1 = nu_df1.query("isTPC == 1").iloc[num]
    #nu_ex3 = nu_df3.query("isTPC == 1").iloc[num]

    #nu_ex1[:]
    
    id = row["G4ID"]

    #print(id)

    r, sr, e = row["run"], row["subrun"], row["evt"]
    
    m1 = (part_df1["run"].values == r)
    m2 = (part_df1["subrun"].values == sr)
    m3 = (part_df1["evt"].values == e)
    m4 = (part_df1["parent"].values == id)
    m = m1 & m2 & m3 & m4

    pdgs = part_df1["pdg"].values[m]

    return len(pdgs)

nu_df1["numD"] = nu_df1.apply(num_d, axis=1)
nu_df1[:2]

# Get the Cosmics in Truth

In [ ]:
#cosmic_df3["interaction_id"] = cosmic_df1["interaction_id"]
cosmic_df1 = part_df1.query("(pdg == 13 or pdg == -13) and interaction_id == -1")
#cosmic_df2 = part_df2.query("pdg == 13 or pdg == -13")
cosmic_df3 = part_df3.query("(pdg == 13 or pdg == -13) and interaction_id == -1")

cosmic_df3[:2]

In [ ]:
plt.hist(cosmic_df3["genT"].values, bins=50, histtype="step", label="Cosmic Muons")

#plt.xlim([0, 10])
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
plt.hist(cosmic_df3["startT"].values, bins=50, histtype="step", label="Cosmic Muons")

#plt.xlim([0, 10])
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:

cosmic_df3["inTPC"] = cosmic_df3.apply(is_AV, axis=1)
cosmic_df3[:2]

In [ ]:
plt.hist(cosmic_df3.query("inTPC == 1")["startT"].values, bins=50, histtype="step", label="Cosmic Muons")
plt.xlabel("True Start Time of TPC interaction [us]", fontsize=14)
#plt.xlim([0, 10])
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
B = np.arange(0, 10, 0.1)

plt.hist(cosmic_df3.query("inTPC == 1 and 0 <= startT <= 10")["startT"].values, bins=B, histtype="step", label="Cosmic Muons")
plt.hist(nu_df3.query("AV == 1 and 0 <= startT <= 10")["startT"].values, bins=B, histtype="step", label="AV")
plt.hist(nu_df3.query("AV == 0 and 0 <= genT <= 10")["genT"].values, bins=B, histtype="step", label="Dirt")
plt.xlabel("Time Since Beam Spill ["+r"$\mu$s]", fontsize=14)
plt.ylabel("Counts/bin/"+str(TOT_POT)+" POT", fontsize=14)
plt.title("SBND MC: True Timing", fontsize=14)
#plt.xlim([0, 10])
plt.yscale("log")
plt.legend()
plt.show()

# Do an event Loop to understand Rates

In [ ]:
evt_tree = inFile["event_tree"]
evt_df = evt_tree.arrays(evt_tree.keys(), library="pd")
evt_df = evt_df.drop_duplicates()
evt_df[:2]

In [ ]:
runs = evt_df["run"].values
subruns = evt_df["subrun"].values

data = {'run': runs,
        'subrun': subruns,
       }

header_df = pd.DataFrame(data)
header_df = header_df.drop_duplicates()

header_df[:2]

In [ ]:
def get_genevt(row):
    r = row["run"]
    sr = row["subrun"]
    g = evt_df.query("run == "+str(r) + " and subrun == "+str(sr))["ngenevt"].values
    if g[-1] != g[0]:
        print("diff ?")
    return g[0]


header_df["genevt"] = header_df.apply(get_genevt, axis=1)
header_df[:3]

In [ ]:
h_2d_nu = ROOT.TH2D("h_2d_nu", "", 5, 0, 5, 5, 0, 5)
h_2d_cosmic = ROOT.TH2D("h_2d_cosmic", "", 5, 0, 5, 5, 0, 5)
print("Initialized histos")

In [ ]:
# check beam window

def is_nu_intime(row):
    av = row["isTPC"]
    if av:
        t = row["endT"]
        if 0<= t <= 2:
            return 1
        else:
            return 0
    else:
        t = row["genT"]
        if 0<= t <= 2:
            return 1
        else:
            return 0        
            
nu_df3["inTime"] = nu_df3.apply(is_nu_intime, axis=1)
nu_df3[:3]

In [ ]:
# check beam window

cosmic_tpc = cosmic_df3.query("inTPC == 1")

def is_cosmic_intime(row): 
    t = row["startT"]
    if 0<= t <= 2:
        return 1
    else:
        return 0
        
            
cosmic_tpc["inTime"] = cosmic_tpc.apply(is_cosmic_intime, axis=1)
cosmic_tpc[:3]

In [ ]:
nu_df3 = nu_df3.query("inTime == 1")
cosmic_tpc = cosmic_tpc.query("inTime == 1")
cosmic_tpc[:2]

In [ ]:

N_cosmic_overall = 0

# Loop over events
for num in range(evt_df.shape[0]):
    if num % 10000 == 0:
        print("Analyzing event", num)
        
    row = evt_df.iloc[num]
    r, sr, e = row["run"], row["subrun"], row["evt"]

    # check cosmic number
    m1 = (cosmic_tpc["run"].values == r)
    m2 = (cosmic_tpc["subrun"].values == sr)
    m3 = (cosmic_tpc["evt"].values == e) 
    m = m1 & m2 & m3
    N_cosmic = len(np.array(cosmic_tpc["pdg"].values)[m])
    #print("number of cosmics", N_cosmic)
    if N_cosmic > 0:
        N_cosmic_overall += 1
    
    # check AV number
    m1 = (nu_df3["run"].values == r)
    m2 = (nu_df3["subrun"].values == sr)
    m3 = (nu_df3["evt"].values == e) 
    m4 = (nu_df3["isTPC"].values == 1)
    m5 = (nu_df3["isTPC"].values == 0)
    
    m_av = m1 & m2 & m3 & m4
    m_dirt = m1 & m2 & m3 & m5

    #print(np.array(nu_df3["pdg"].values)[m_av])
    
    N_av = len(np.array(nu_df3["pdg"].values)[m_av])
    N_dirt = len(np.array(nu_df3["pdg"].values)[m_dirt])

    #if num > 5:
    #    break
    
    #print("number of AV", N_av)
    #print("number of Dirt", N_dirt)
    h_2d_cosmic.Fill(N_cosmic, N_av)
    h_2d_nu.Fill(N_dirt, N_av)
    
print("Done!")

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kRainBow)
h_2d_nu.SetStats(0)
h_2d_nu.SetTitle("")

c = ROOT.TCanvas("c", "c", 700, 500)
h_2d_nu.GetXaxis().SetTitle("Number of Dirt #nu")
h_2d_nu.GetYaxis().SetTitle("Number of AV #nu")
#h_2d_nu.GetZaxis().Setlog()
ROOT.gStyle.SetPaintTextFormat("1.1e")
h_2d_nu.Draw("Colz TEXT")
ROOT.gPad.SetLogz()
c.Draw()

In [ ]:
h_2d_cosmic.SetStats(0)
h_2d_cosmic.SetTitle("")
#ROOT.gPad.SetLogz()
c = ROOT.TCanvas("c", "c", 700, 500)
h_2d_cosmic.GetXaxis().SetTitle("Number of Cosmic")
h_2d_cosmic.GetYaxis().SetTitle("Number of AV #nu")
#h_2d_nu.GetZaxis().Setlog()
ROOT.gStyle.SetPaintTextFormat("1.1e")
h_2d_cosmic.Draw("Colz TEXT")
ROOT.gPad.SetLogz()
c.Draw()

In [ ]:

def get_nu_rate(h):
    N_tot = 0
    N_nu = 0
    for i in range(1, h.GetNbinsX()+1):
        for j in range(1, h.GetNbinsY()+1):
            if j >= 2:
                N_nu += h.GetBinContent(i, j)
            N_tot += h.GetBinContent(i, j)
            
    return ((1.0*N_nu)/N_tot)


nu_r_d = get_nu_rate(h_2d_nu)
nu_r_c = get_nu_rate(h_2d_cosmic)

In [ ]:
print(nu_r_d)
print(nu_r_c)

In [ ]:
def get_other_rate(h):
    N_tot = 0
    N_other = 0
    for i in range(1, h.GetNbinsX()+1):
        for j in range(1, h.GetNbinsY()+1):
            if i >= 2:
                N_other += h.GetBinContent(i, j)
            N_tot += h.GetBinContent(i, j)
            
    return ((1.0*N_other)/N_tot)


cosmic_r = get_other_rate(h_2d_cosmic)
dirt_r = get_other_rate(h_2d_nu)

print(cosmic_r)
print(dirt_r)

In [ ]:
# Determin the correct event normalization

E_tot = np.sum(header_df["genevt"].values)

E_kept = evt_df.shape[0]

print("E_kept", E_kept)
print("E_tot", E_tot)

In [ ]:
E_lost = int(E_tot - E_kept)

h_2d_cosmic.SetBinContent(1, 1, h_2d_cosmic.GetBinContent(1, 1)+E_lost)
h_2d_nu.SetBinContent(1, 1, h_2d_nu.GetBinContent(1, 1)+E_lost)
print("Added Lost Events")

In [ ]:
c = ROOT.TCanvas("c", "c", 700, 500)
h_2d_nu.GetXaxis().SetTitle("Number of Dirt #nu")
h_2d_nu.GetYaxis().SetTitle("Number of AV #nu")
#h_2d_nu.GetZaxis().Setlog()
ROOT.gStyle.SetPaintTextFormat("1.1e")
h_2d_nu.Draw("Colz TEXT")
ROOT.gPad.SetLogz()
c.Draw()

In [ ]:
c = ROOT.TCanvas("c", "c", 700, 500)
h_2d_cosmic.GetXaxis().SetTitle("Number of Cosmic")
h_2d_cosmic.GetYaxis().SetTitle("Number of AV #nu")
#h_2d_nu.GetZaxis().Setlog()
ROOT.gStyle.SetPaintTextFormat("1.1e")
h_2d_cosmic.Draw("Colz TEXT")
ROOT.gPad.SetLogz()
c.Draw()

In [ ]:
nu_r_d = get_nu_rate(h_2d_nu)
nu_r_c = get_nu_rate(h_2d_cosmic)
cosmic_r = get_other_rate(h_2d_cosmic)
dirt_r = get_other_rate(h_2d_nu)

print(nu_r_c)
print(nu_r_d)
print(cosmic_r)
print(dirt_r)

In [ ]:
print(N_cosmic_overall/E_kept)
print(N_cosmic_overall/E_tot)